statistica: numero di pixel nell'originale, numero di stati nel modello
minimizzato, tempi di model-checking con voxlogica sull'originale vs.
con graphlogica sul LTS minimizzato (anche se non possiamo visualizzare
il risultato, ma sara' future work).


In [1]:
import subprocess 
import time
import glob
import os
from pathlib import Path
import shutil
import pandas as pd

def run(obj):
    start = time.perf_counter()
    result = subprocess.run(obj["args"],capture_output=True,text=True)
    return { "delta": time.perf_counter() - start, "label": obj["label"],"output": result.stdout,"error": result.stderr, "return_code": result.returncode }

converter_exe_rel="../../../releases/GraphLogicA_0.4_win-x64/GraphLogicA.exe"
converter_exe = Path(converter_exe_rel).absolute().as_posix()
minimizer_exe = shutil.which("ltsconvert")
output="output"
shutil.rmtree(output)
os.makedirs(output,exist_ok=True)
images = glob.glob("test-images/*.png")

def mk_df(results,delta_label):
    return pd.DataFrame(results).set_index("label").rename(columns={"delta": delta_label}).drop(columns=["output","error","return_code"])

In [2]:
def converter(image):
    path = Path(image)
    label = path.name
    o_path = Path(output)
    s_path = path.with_suffix(".aut").name
    d_path = o_path.joinpath(s_path)
    return { "args": [converter_exe,"--convert",path.as_posix(),d_path.as_posix()], "label": label }
    #return (run(label,args))

converter_df = mk_df([ run(converter(image)) for image in images ],"conversion")


In [ ]:
def minimizer(image):
    path = Path(image)
    label = path.name
    o_path = Path(output)
    s_path = path.with_suffix(".aut").name
    d_path = o_path.joinpath(s_path)
    m_path = o_path.joinpath(Path(path.with_suffix("").name + "_min").with_suffix(".aut"))
    return { "args": [minimizer_exe,"-ebranching-bisim",d_path.as_posix(),m_path.as_posix()], "label": label }

minimizer_df = mk_df([ run(minimizer(image)) for image in images ],"minimization")


In [ ]:
df = converter_df.join(minimizer_df)
df

,conversion,minimizer
label,,
MMtest1.png,4.698367,0.137208
